## Solve linear equations with mip

In [1]:
from mip import *
m = Model()

x = m.add_var(var_type=CONTINUOUS)
y = m.add_var(var_type=CONTINUOUS)

m += 120 * x + 150 * y == 1440
m += x + y == 10

m.optimize()

if m.num_solutions:
    print(x.name, x.x, y.name, y.x)

var(0) 2.000000000000002 var(1) 7.999999999999998


## Solve LP(Linear Optimization Problem)

### - with pulp

In [2]:
import pulp

problem = pulp.LpProblem('LP', pulp.LpMaximize)

x = pulp.LpVariable('x', cat='Continuous')
y = pulp.LpVariable('y', cat='Continuous')

problem += 1 * x + 3 * y <= 30
problem += 2 * x + 1 * y <= 40
problem += x >= 0
problem += y >= 0
problem.objective = x + 2 * y

status = problem.solve()

print('Status:', pulp.LpStatus[status])
print('x=', x.value(), 'y=', y.value(), 'obj=', problem.objective.value())

Status: Optimal
x= 18.0 y= 4.0 obj= 26.0


### - with mip

In [3]:
from mip import *
m = Model()

x = m.add_var(name='x', var_type=CONTINUOUS)
y = m.add_var(name='y', var_type=CONTINUOUS)

m += 1 * x + 3 * y <= 30
m += 2 * x + 1 * y <= 40
m += x >= 0
m += y >= 0

m.objective = maximize(x + 2 * y)

m.optimize()

if m.num_solutions:
    print(x.name, x.x, '\n', y.name, y.x, '\nsolution', m.objective.x)

x 18.0 
 y 4.000000000000001 
solution 26.0


In [4]:
m.status

<OptimizationStatus.OPTIMAL: 0>

In [5]:
print(list(mip.OptimizationStatus))

[<OptimizationStatus.ERROR: -1>, <OptimizationStatus.OPTIMAL: 0>, <OptimizationStatus.INFEASIBLE: 1>, <OptimizationStatus.UNBOUNDED: 2>, <OptimizationStatus.FEASIBLE: 3>, <OptimizationStatus.INT_INFEASIBLE: 4>, <OptimizationStatus.NO_SOLUTION_FOUND: 5>, <OptimizationStatus.LOADED: 6>, <OptimizationStatus.CUTOFF: 7>, <OptimizationStatus.OTHER: 10000>]


## Solve LP of larger scale

In [6]:
import pandas as pd
from mip import *

s_df = pd.read_csv('stocks.csv')
r_df = pd.read_csv('requires.csv')
g_df = pd.read_csv('gains.csv')

s_df

,m,stock
0,m1,35
1,m2,22
2,m3,27


In [7]:
r_df

,p,m,require
0,p1,m1,2
1,p1,m2,0
2,p1,m3,1
3,p2,m1,3
4,p2,m2,2
5,p2,m3,0
6,p3,m1,0
7,p3,m2,2
8,p3,m3,2
9,p4,m1,2


In [8]:
g_df

,p,gain
0,p1,3
1,p2,4
2,p3,4
3,p4,5


In [9]:
P = g_df['p'].tolist()
M = s_df['m'].tolist()

In [10]:
stock = {r.m: r.stock for r in s_df.itertuples()}
stock

{'m1': 35, 'm2': 22, 'm3': 27}

In [11]:
gain = {r.p: r.gain for r in g_df.itertuples()}
gain

{'p1': 3, 'p2': 4, 'p3': 4, 'p4': 5}

In [12]:
require = {(r.p, r.m): r.require for r in r_df.itertuples()}
require

{('p1', 'm1'): 2,
 ('p1', 'm2'): 0,
 ('p1', 'm3'): 1,
 ('p2', 'm1'): 3,
 ('p2', 'm2'): 2,
 ('p2', 'm3'): 0,
 ('p3', 'm1'): 0,
 ('p3', 'm2'): 2,
 ('p3', 'm3'): 2,
 ('p4', 'm1'): 2,
 ('p4', 'm2'): 2,
 ('p4', 'm3'): 2}

In [13]:
m = Model()

x = m.add_var_tensor((len(P),), name='x', var_type=CONTINUOUS)
x[3].name

'x_3'

In [14]:
m.objective = maximize(xsum(x_i * gain[p_i] for x_i, p_i in zip(x, P)))
for m_j in stock.keys():
    m += xsum([require[p_i, m_j] * x_i for x_i, p_i in zip(x, P)]) <= stock[m_j]
    
m.optimize()

<OptimizationStatus.OPTIMAL: 0>

In [15]:
if m.num_solutions:
    for x_i in x:
        print(x_i.name, ': ', x_i.x)
        
    print('solution: ', m.objective.x)

x_0 :  12.142857142857142
x_1 :  3.571428571428571
x_2 :  7.428571428571429
x_3 :  0.0
solution:  80.42857142857143
